In [34]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline
torch.manual_seed(1)

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

In [6]:
def findFiles(path): return glob.glob(path)

In [18]:
filePath = os.path.join(os.getcwd(),'GenCharRnn_data/names/*.txt')
fileWithNames = findFiles(filePath)

In [31]:
## Helper - boilerplate code

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

category_lines = {}
all_categories = []
for f in fileWithNames:
    category = os.path.splitext(f)[0].split('/')[-1]
    all_categories.append(category)
    with open(f,'r') as names:
        category_lines[category] = [n.strip() for n in names.readlines()]
        
n_categories = len(all_categories)

In [36]:
# category_dim = n_categories
class GenRNN(nn.Module):
    def __init__(self,inp_dim,out_dim,hidden_dim,category_dim):
        super(GenRNN, self).__init__()
        self.i2o = nn.Linear(inp_dim+hidden_dim+category_dim,out_dim)
        self.i2h = nn.Linear(inp_dim+hidden_dim+category_dim,hidden_dim)
        self.o2o = nn.Linear(out_dim,out_dim)
        self.dropout = nn.Dropout(p=0.1) # isn't this very low?
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self,inp,hidden_prev,category):
        combined = torch.cat((inp,hidden,category))
        out1 = self.i2o(combined)
        hidden = self.i2h(combined)
        out_combined = torch.cat((output,hidden))
        out2 = self.softmax(self.dropout(self.o2o(out_combined)))
        
        return out2, hidden